In [2]:
!pip install langchain openai

# Getting Started with Chains

In [13]:
import inspect
import re

from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain

In [4]:
OPENAI_API_KEY = ""

In [5]:
llm = OpenAI(
    temperature=0,
    openai_api_key=OPENAI_API_KEY
    )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## What are chains anyway?

**Definition**: Chains are one of the fundamental building blocks of this lib (as you can guess!).

The official definition of chains is the following:


> A chain is made up of links, which can be either primitives or other chains. Primitives can be either prompts, llms, utils, or other chains.


So a chain is basically a pipeline that processes an input by using a specific combination of primitives. Intuitively, it can be thought of as a 'step' that performs a certain set of operations on an input and returns the result. They can be anything from a prompt-based pass through a LLM to applying a Python function to an text.

Chains are divided in three types: Utility chains, Generic chains and Combine Documents chains.

1. Utility Chains: chains that are usually used to extract a specific answer from a llm with a very narrow purpose and are ready to be used out of the box.
2. Generic Chains: chains that are used as building blocks for other chains but cannot be used out of the box on their own.

Let's take a peek into what these chains have to offer!

### Utility Chains

Let's start with a simple utility chain. The `LLMMathChain` gives llms the ability to do math. Let's see how it works!

#### Pro-tip: use `verbose=True` to see what the different steps in the chain are!

In [6]:
llm_math = LLMMathChain(llm=llm, verbose=True)

llm_math.invoke("What is 76 raised to the .878 power?")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 76 raised to the .878 power?```text
76**0.878
```
...numexpr.evaluate("76**0.878")...

Answer: 44.80787354590261
> Finished chain.


{'question': 'What is 76 raised to the .878 power?',
 'answer': 'Answer: 44.80787354590261'}

In [7]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [8]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

llm_chain.invoke("What is 13 raised to the .3432 power?")

{'question': 'What is 13 raised to the .3432 power?',
 'text': '\n\n13 raised to the .3432 power is approximately 2.999.'}

**Insight**: _by using prompts intelligently, we can force the llm to avoid common pitfalls by explicitly and purposefully programming it to behave in a certain way._

Another interesting point about this chain is that it not only runs an input through the llm but it later compiles Python code. Let's see exactly how this works.

In [9]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



So we can see here that if the llm returns Python code we will compile it with a Python REPL* simulator. We now have the full picture of the chain: either the llm returns an answer (for simple math problems) or it returns Python code which we compile for an exact answer to harder problems. Smart!

Also notice that here we get our first example of **chain composition**, a key concept behind what makes langchain special. We are using the `LLMMathChain` which in turn initializes and uses an `LLMChain` (a 'Generic Chain') when called. We can make any arbitrary number of such compositions, effectively 'chaining' many such chains to achieve highly complex and customizable behaviour.

Utility chains usually follow this same basic structure: there is a prompt for constraining the llm to return a very specific type of response from a given query. We can ask the llm to create SQL queries, API calls and even create Bash commands on the fly.

### Generic chains

There are only three Generic Chains in langchain.

In [14]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [15]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [16]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

Now we want to use our chain to clean an input text and then paraphrase the input in a specific style, say a poet or a policeman.

First we will build the prompt template:

In [17]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate.from_template(template)


And next, initialize our chain:

In [19]:
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

Finally, we need to combine them both to work as one integrated chain. For that we will use `SequentialChain` which is our third generic chain building block.

In [20]:
sq_chain = clean_extra_spaces_chain | style_paraphrase_chain
# sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

Our input is the langchain docs description of what chains are but dirty with some extra spaces all around.

In [21]:
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

We are all set. Time to get creative!

In [22]:
sq_chain.invoke({'text': input_text, 'style': 'a 90s rapper'})
# count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

{'text': '\nChains allow us to combine multiple\n\n\ncomponents together to create a single, coherent application.\n\nFor example, we can create a chain that takes user input,       format it with a PromptTemplate,\n\nand then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by\n\n\ncombining chains with other components.\n',
 'style': 'a 90s rapper',
 'output_text': '\nChains allow us to combine multiple\ncomponents together to create a single, coherent application.\nFor example, we can create a chain that takes user input, format it with a PromptTemplate,\nand then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by\ncombining chains with other components.\n',
 'final_output': "Yo, check it - chains be the key to makin' one dope app. Like, we can hook up a chain that takes what the user says, jazz it up with a PromptTemplate, and then send it to an